In [ ]:
import pandas as pd
import numpy as np

In [ ]:
loans=

In [ ]:
#DATA PREPROCESSING

"""
for column in wine_review.columns:
    print(loan[column].dtype)
    if loan[column].dtype == type(object):
        le = LabelEncoder()
        # le.fit(loan[column].astype(str))
       # row = le.tranform(loan[column])
"""

#x are entire dataset fields short of the one we are predicting(Interest rate), dropped 
X=loan.drop("Int_Rate",axis=1)
y=loan["Int_Rate"]
print(X.shape,y.shape)

In [ ]:
#import deeplearning modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

#split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
#Compile the model - CHANGE EPOCHS
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)


In [ ]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
#prediction testing
encoded_predictions = model.predict_classes(X_test_scaled[:5])

prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f"Predicted Interest Rate: {prediction_labels}")
print(f"Actual Interest Rate: {list(y_test[:5])}")

In [ ]:
# Save the model
model.save("loan_model_trained.h5")